# Do local bank branches affect home mortgages?

#### An exploratory data analysis using Urban Institute and U.S. Census data

Author: Kathryn Hurchla,
Updated: Sept. 14, 2021

- Data: I'm focusing on mortgage disclosure data, and have begun to link it with banking locations from the FDIC.
    - Home Mortgage Disclosure Act Neighborhood Summary Files: Census Tract Level, Urban Institute


- I plan to take a statistical investigation approach with my project, and potentially mapping as well.
    - Correlations between lending patterns and presense or absence of local banks/branches


- Toolset
    - I'm using Python for data analysis, and 
    - I plan to either use Ploty or other Python libraries for charts, or to take the opportunity to try using Observable for a basic line or bar chart.
    - The end product may be static, e.g. a data heavy report or web journalism format
    - Alternately, a more advanced interactive using Python Dash


- My inspiration comes from my own experience living in a neighborhood that lacked a bank for 30 years, and recently gained a regionally based PA bank in the commercial corridor that I've enjoyed the customer service and 'family' touch banking with.
    - Also building on my broader ongoing real estate and housing affordability analysis


- I would like to tell a story with the data about the impact of gaps in core services/financial accessibility.
    - A data informed update within the context of the lineage of redlining and reverse-redlining
    - Potential audiences are local officials/representatives in:
        - government
        - business professional organizations that would include potential banks to attract
        - community development corporations

In [1]:
import plotly.express as px
import pandas as pd

In [2]:
#Load the csv file on the most recent 2019 year available of Home Mortgage Disclosure Act Neighborhood Summary Files: Census Tract Level
#File is already downloaded from the Urban Data Catalog, and saved locally 
#define specific columns needed to read in since it's a large file
col_list_mrtg = ["geo2010", # US Census tract ID (ssccctttttt) per definition in Urban-HMDA_neighborhood_data_codebook
            'geo2010_11', # corrected for missing leading zeros, all have 11 digits now
            'CensusTractID', # last 6 digits of geo2010_11
            'StateID', # first 2 digits of geo2010_11
            'CountyID', # mid 3 digits of geo2010_11, following StateID digits
            #"owner_purchase_originations", 
            'purch_orig', 
            "median_loan_amount", 
            "median_income", 
            "income_avail", 
            "race_avail", 
            "race_income_avail", 
            "invalid_geo"]
#Load the specified columns in the col_list variable above
df_mrtg = pd.read_csv("./data/hmda_tract_2019.csv", usecols=col_list, low_memory=False)
#File is presorted by geo2010 variable; also sort triggered an error because the geo2010 field has charachter type 'TypeError: '<' not supported between instances of 'str' and 'int''
#df = df.sort_values(by='geo2010')
#display the dataframe
df_mrtg

NameError: name 'col_list' is not defined

### Taking a glimpse at the distribution of the mortgage data

Along with availability of income (high/low indicators are in dataset also), and race

What other stories might I tell in this dataset without linking with the financial institution locations?

In [ ]:
# Exploratory combined statistical Distplot of mortgage data
fig = px.scatter_matrix(df_mrtg,
#specify columns to show, colors
    dimensions=["purch_orig", "median_loan_amount", #"income_avail", "race_avail", 
                "race_income_avail"],
    title='Distribution of Median Loan Amount with Availability of Borrowers Income and Race')

fig.show(renderer='browser')

In [ ]:
#Load the csv file on the list of all bank institutions and branches with Census CBSA codes
#File is already downloaded from the FDIC, and saved locally 
#define specific columns needed to read in since it's a large file
col_list_bks = ["BKCLASS", #Institution Class: a list of codes can be found in the OFFICES2_DEFINITIONS file
            'CBSA', #US Census Core Based Statistical Area Name (Branch)
            'CBSA_NO', #CBSA number 
            "CITY", 
            "COUNTY", 
            "CSA", #US Census Combined Statistical Area Name (Branch)
            "CSA_NO", #CSA number
            "NAME", #Institution Name
            "OFFNAME", #Office Name
            "OFFNUM", #Office number
            "SERVTYPE"] #Service Type Code: a list of codes can be found in the OFFICES2_DEFINITIONS file 
#Load the specified columns in the col_list variable above
#Note default UTF-8 was resulting in error: UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 2: invalid continuation byte
#Therefore solution to explicitly specific latin-1 encoding was added as per stackoverflow solutions
df_bks = pd.read_csv("./data/OFFICES2_ALL.CSV", encoding='latin-1', usecols=col_list_bks)
#File is presorted by geo2010 variable; also sort triggered an error because the geo2010 field has charachter type 'TypeError: '<' not supported between instances of 'str' and 'int''
#df = df.sort_values(by='geo2010')
#display the dataframe
df_bks

### Linking structure to analyze bank locations and mortgage disclosure data

Link  [Bank (df_bks)].[CBSA_NO] 


to [crosswalk (df_cx].[cbsacode]


and to [crosswalk (df_cx].[fipsstatecode_2]+[fipscountycode_3]


to [mortgages (df_mrtg)].[StateID]+[CountyID]

### Challenges

1. Difficulty isolating only owner occupied home loans
    - The variable I had to use may include investment properties/rentals where the person who got a mortgage is not using it as their own home.
    - It still provides a high level view of property lending including investment interests.
    - 'owner_purchase_originations' variable cited in code book is not found in the 2019 data file.
    - Extending beyond a single year, data are in separate files for each year.


2. Understanding Census codes!!
    - This was a helpful resource: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html
    - and this about Core-Based Statistical Areas (CBSA) https://www.census.gov/topics/housing/housing-patterns/about/core-based-statistical-areas.html
    - "The full GEOID for many levels of geography combines both the FIPS codes and Census Bureau codes. For example, census tracts, block groups and census blocks nest within state and county; therefore, the GEOIDs for each of these geographic areas contains both the state and county FIPS codes, in which they nest."
    - "For example, census tracts, block groups and census blocks nest within state and county; therefore, the GEOIDs for each of these geographic areas contains both the state and county FIPS codes, in which they nest."


3. Hierarchy and nested codes:
    - United States
        - Region
            - Division
                - State
                    - County
                        - County subdivision
                            - Place (or part)
                                - Census tract (or part)
                                    - Block group (or part)
                                         - Census block


4. Requirement to use a crosswalk to link to bank locations because 
    - Different Census geographic identifiers were used by the different data sources.
    - Found an existing resource to bridge Census Core-Based Statistical Area (CBSA) to Federal Information Processing Series (FIPS) County Crosswalk from the National Bureau of Economic Research.
    - In offices file (banks) some CSBA numbers are '0'


5. May only be able to link banks to the county level (state + county) with this crosswalk
    - Does not tell the story of urban neighborhoods in some counties in contrast to more rural counties
    - May be able to drill further into CBSA areas in metropolitan areas through mapping


5. Some pesky data issues in the FDIC bank file: 
    - having to do with encoding as UTF-8 or latin... so I'll need to verify the data a bit further to confirm the solution and that it's clean.
    - Missing CBSA codes: 9,100 records list '0' out of 84,076 total bank location records (about 10%). I can't see any similarity in these records versus the others and some appear to be a very close location to others that do have a CBSA listed, so it seems to be likely missing data in the file. These have '0' CSA also.
    - Note the valid CBSA codes in this file have 5 digits, not 1 ('0')

### Cited & Referenced

Federal Deposit Insurance Corporation (FDIC), accessed 8/31/2021
1. Current list of all institutions (3.68 MB)
2. Current locations for all institutions (branches and main offices) (7.50 MB)
https://www7.fdic.gov/idasp/warp_download_all.asp

US Census Glossary, accessed 8/31/2021,
https://www.census.gov/programs-surveys/geography/about/glossary.html#par_textimage_13

Home Mortgage Disclosure Act Neighborhood Summary Files: Census Tract Level, Urban Institute, accessed 8/31/2021,
https://datacatalog.urban.org/dataset/home-mortgage-disclosure-act-neighborhood-summary-files-census-tract-level

National Census-tract level summary file of Home Mortgage Disclosure Act (HMDA) indicators, Github repository, Urban Institute, accessed 8/31/2021,
https://github.com/UI-Research/hmda-neighborhood

Census Core-Based Statistical Area (CBSA) to Federal Information Processing Series (FIPS) County Crosswalk, National Bureau of Economic Research, accessed 8/31/2021,
https://www.nber.org/research/data/census-core-based-statistical-area-cbsa-federal-information-processing-series-fips-county-crosswalk